In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
import time
import pandas as pd
import warnings
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
warnings.filterwarnings("ignore") 

In [2]:
driver = webdriver.Chrome("C:/Users/wnwn6/chromedriver.exe")
driver.get('https://www.musinsa.com/app/goods/1571501')
time.sleep(3)

In [3]:
#user, gender,height,weight,item, size,content, evaluation(size_eval,bright_eval,color_eval,thick_eval)
user_list = []
gender_list =[]
height_list = []
weight_list =[]
item_list = []
size_list = []
star_list= []
content_list = []
size_eval_list =[]
bright_eval_list =[]
color_eval_list =[]
thick_eval_list =[]

#함수정의
def get_content(driver):
    #함수안에 html, soup 넣어놔야 페이지 넘어가서 바르게 긁어옴, 밖에 빼놓으면 첫페이지만 여러번 긁어진다.
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')
    for i in range(10):
    
        profile_before = soup.find_all('p','review-profile__body_information')
        profile_after = profile_before[i].text.split(',')
        try:
            gender = profile_after[0]
            height = profile_after[1]
            weight = profile_after[2]
        except:
            gender = ''
            height = ''
            weight = ''
#user :LV 2 뉴비_95f88e16

           
#item: #/테이퍼드 히든 밴딩 크롭 슬랙스 [더스티 베이지]
#size :
#content

        try:
            user = soup.find_all('p','review-profile__name')[i].text
            item = soup.find_all('a','review-goods-information__name')[i].text
            # '/n' 없애고 추출하기
            size = soup.find_all('span', 'review-goods-information__option')[i].text.strip().replace('\n','')
            content = soup.find_all('div','review-contents__text')[i].text
        except:
            user = ''
            item = ''
            size = ''
            content = ''
            
#star
#->별 5개일때: //*[@id="reviewListFragment"]/div[4]/div[3]/span/span/span
#->별 4개일때: //*[@id="reviewListFragment"]/div[5]/div[3]/span/span/span
        stars = driver.find_elements_by_xpath('//*[@id="reviewListFragment"]/div['+str(i+1)+']/div[3]/span/span/span')
        try:
            for j in stars:
                a =j.get_attribute('style')
                if a[7:9]=='20':
                    star = 1
                elif a[7:9]=='40':
                    star = 2
                elif a[7:9]=='60':
                    star = 3
                elif a[7:9]=='80':
                    star = 4
                else:
                    star = 5
        except:
            star = ''
      

    #evaluation
        evaluation = soup.find_all('div', 'review-evaluation')
        try:
            size_eval = evaluation[i].find_all('span')[0].text
            bright_eval = evaluation[i].find_all('span')[1].text
            color_eval = evaluation[i].find_all('span')[2].text
            thick_eval = evaluation[i].find_all('span')[3].text
        except:
            size_eval = ''
            bright_eval = ''
            color_eval = ''
            thick_eval = ''

#user,gender,height,weight,item, size,content,star,size_eval,bright_eval,color_eval,thick_eval
        
        user_list.append(user)
        gender_list.append(gender)
        height_list.append(height)
        weight_list.append(weight)
        item_list.append(item)
        size_list.append(size)
        content_list.append(content)
        star_list.append(star)
        size_eval_list.append(size_eval)
        bright_eval_list.append(bright_eval)
        color_eval_list.append(color_eval)
        thick_eval_list.append(thick_eval)

In [4]:
#버튼 누르기 함수정의
def move_next(driver):    
    for i in range(4):
        get_content(driver)
        #페이지 2,3,4 넘어가기
        driver.find_element_by_css_selector('#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div > a:nth-child(' + 
                                            str(int(4) + int(i)) + ')').send_keys(Keys.ENTER)
        time.sleep(2)
    #그다음 화살표'>'버튼누르기: (6,7,8...)있는 페이지로 넘어가기
    get_content(driver)
    
def move_arrow(driver):
    driver.find_element_by_css_selector('#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div > a.fa.fa-angle-right.paging-btn.btn.next').send_keys(Keys.ENTER)

In [5]:
for i in range(200):
     try:
        move_next(driver)
        move_arrow(driver)
    #move_next(driver)
     except:
        time.sleep(2)
    #move_next(driver)

df = pd.DataFrame({'user':user_list, 
                   'gender':gender_list,
                   'height':height_list ,
                   'weight':weight_list,
                   'item':item_list, 
                   'size':size_list,
                   'star':star_list,
                   'content':content_list,
                   'size_eval':size_eval_list,
                   'bright_eval':bright_eval_list,
                   'color_eval':color_eval_list,
                   'thick_eval':thick_eval_list})

In [6]:
df

,user,gender,height,weight,item,size,star,content,size_eval,bright_eval,color_eval,thick_eval
0,LV 3 탠현,남성,160cm,38kg,빅 트위치 루즈핏 후드 티셔츠 네이비,S,5,기대했던거보다 너무이쁘고 사기 잘한거 같에요,보통이에요,어두워요,선명해요,보통이에요
1,LV 3 여름이되면재채기가너무심해져,남성,188cm,100kg,빅 트위치 루즈핏 후드 티셔츠 네이비,XL,5,"사이즈는 상세페이지에 작성된 내용과 같고, 실착도 딱 좋네요. 색깔도 우중충한 네이...",보통이에요,보통이에요,보통이에요,두꺼워요
2,LV 3 뉴비_65ac65a,남성,170cm,64kg,빅 트위치 루즈핏 후드 티셔츠 네이비,M,5,생각하던 핏이 딱 나와준 옷이에요 옷 두께는 좀 얇은 느낌이 들어요,커요,보통이에요,선명해요,보통이에요
3,LV 5 여누하,남성,173cm,73kg,빅 트위치 루즈핏 후드 티셔츠 그레이,XL,5,정말로 좋아하는 스타일의 옷이에요 데일리 또는 스포츠레저 등 다양하게 입을수 있구 ...,커요,밝아요,선명해요,두꺼워요
4,LV 3 930615,남성,165cm,54kg,빅 트위치 루즈핏 후드 티셔츠 네이비,M,5,"무진장 블프하기 며칠 전에 샀지만,, 난 괜찮아,, 그동안 많이 입었으니까 ,,^^...",보통이에요,보통이에요,보통이에요,보통이에요
...,...,...,...,...,...,...,...,...,...,...,...,...
2005,LV 4 뉴비_83b514d90e8f,여성,160cm,60kg,빅 트위치 루즈핏 후드 티셔츠 블랙,M,5,귀엽고 재질도 도톰한게 딱 맘에 들어요 오래입을수있을거같아요,커요,보통이에요,보통이에요,두꺼워요
2006,LV 3 뉴비_6527c64f,여성,158cm,60kg,빅 트위치 루즈핏 후드 티셔츠 블랙,S,5,공용이라 사이즈 완전 넉넉하네용158/60 s 입니다!대만족이예요,커요,보통이에요,보통이에요,보통이에요
2007,LV 3 킄크루킄크,남성,173cm,63kg,빅 트위치 루즈핏 후드 티셔츠 네이비,M,5,색도 너무 이쁘게 나왔고 편하게 입기 좋은거 같아요.,보통이에요,보통이에요,보통이에요,보통이에요
2008,LV 6 울트라상지니,남성,178cm,72kg,빅 트위치 루즈핏 후드 티셔츠 네이비,L,5,기모 아니라서 오히려 좋음. 진짜 이쁘고 모자도 커서 좋음,보통이에요,보통이에요,선명해요,보통이에요


In [7]:
df.to_csv('hood5.csv')
# hood1 = top1 후드제품 